# Special Methods
In this section we will learn about a variety of instance methods that are reserved by Python, which affect an object's high level behavior and its interactions with operators. These are known as special methods. `__init__` is an example of a special method; recall that it controls the process of creating instances of a class. Similarly, we will see that `__add__` controls the behavior of an object when it is operated on by the `+` symbol, for example. Learning to leverage special methods will enable us to design elegant and powerful classes of objects,

In general, the names of special methods take the form of `__<name>__`, where the two underscores proceed and succeed the name. Accordingly, special methods can also be referred to as "dunder" (double-underscore) methods. 


This section is not meant to be a comprehensive treatment of special methods, which would require us to reach beyond our desired level of sophistication. The [official Python documentation](https://docs.python.org/3/reference/datamodel.html#special-method-names) provides a rigorous but somewhat inaccessible treatment of special methods. [Dive into Python 3](http://www.diveintopython3.net/special-method-names.html) has an excellent appendix on special methods. It is strongly recommended that readers consult this resource.

In [28]:
class a:
    def __div__(self, other) : return 1
    def __floordiv__(self, other) : return 0

In [29]:
a() // 22

0

## Interfacing with Mathematical Operators
The following special methods control how an object interacts with `+`, `*`, `**`, and other mathematical operators. 

|Method| Signature | Explanation |
|---|---|---|
|Add|`__add__(self, other)`|`x + y` invokes `x.__add__(y)`|
|Subtract|`__sub__(self, other)`|`x - y` invokes `x.__sub__(y)`|
|Multiply|`__mul__(self, other)`|`x * y` invokes `x.__mul__(y)`|
|Divide|`__truediv__(self, other)`|`x / y` invokes `x.__truediv__(y)`|
|Power|`__pow__(self, other)`|`x ** y` invokes `x.__pow__(y)`|

You may be wondering why division has the peculiar name `__truediv__`, whereas the other operators have more sensible names. This is an artifact of the transition from Python 2 to Python 3; [the default integer-division was replaced by float-division](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Basic_Objects.html#Number-Types), and thus `__div__` was replaced by `__truediv__` for the sake of 2-3 compatibility.     

In [14]:
strike("ryan hello ")

'̶r̶y̶a̶n̶ ̶h̶e̶l̶l̶o̶ '

In [13]:
def strike(text):
    return ''.join('\u0336{}'.format(c) for c in text)

## 

## Links to Official Documentation

- [Special Methods](https://docs.python.org/3/reference/datamodel.html#special-method-names)